In [1]:
import torch
from torchvision import datasets, transforms
import helper
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from network import Network

In [2]:
import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convolutional layer (sees 64x64x3 image tensor)
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        # convolutional layer (sees 32x32x16 tensor)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        # convolutional layer (sees 16x16x32 tensor)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        # conv layer 4 sees 8X8X64
        self.conv4 = nn.Conv2d(64,128,3,padding=1)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        # linear layer (64 * 4 * 4 -> 500)
        self.fc1 = nn.Linear(128 * 4 * 4, 128)
        # linear layer (500 -> 10)
        self.fc2 = nn.Linear(128, 12)
        # dropout layer (p=0.25)
        self.dropout = nn.Dropout(0.25)
        
    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        #print(x.shape)
        x = self.conv1(x)
        #print(x.shape)
        x = F.relu(x)
        #print(x.shape)
        x = self.pool(x)
        #print(x.shape)
        #x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        #print(x.shape)
        # flatten image input
        x = x.view(-1, 128 * 4 * 4)
        #x = x.view(x.size(0), -1)
        #print(x.shape)
        # add dropout layer
        x = self.dropout(x)
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add 2nd hidden layer, with relu activation function
        x = self.fc2(x)
        # output layer
        x = F.log_softmax(x, dim=1)
        return x
# create a complete CNN
model = Net()
print(model)


Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2048, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=12, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)


In [3]:
checkpoint = torch.load('../models/model_13.pt')

In [4]:
checkpoint.keys()

odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'conv3.weight', 'conv3.bias', 'conv4.weight', 'conv4.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'])

In [5]:
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [6]:
model.state_dict()

OrderedDict([('conv1.weight', tensor([[[[ 0.0271, -0.0027, -0.1272],
                        [ 0.1524, -0.0889,  0.1360],
                        [ 0.0345,  0.0092,  0.1147]],
              
                       [[ 0.1528,  0.1611, -0.1058],
                        [ 0.0599, -0.0845,  0.1622],
                        [-0.0602, -0.0725, -0.1861]],
              
                       [[-0.1997, -0.1797, -0.0623],
                        [-0.0053,  0.1234, -0.0892],
                        [ 0.1433, -0.1123,  0.1563]]],
              
              
                      [[[ 0.0508,  0.0832, -0.0052],
                        [ 0.1249, -0.1160,  0.0128],
                        [ 0.1693, -0.0017, -0.0531]],
              
                       [[ 0.1770,  0.1537,  0.0233],
                        [-0.1222,  0.0535,  0.1435],
                        [ 0.0556,  0.1568, -0.0674]],
              
                       [[-0.0872, -0.0690, -0.0573],
                        [ 0.0086,  0.175

In [7]:
model.eval()

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2048, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=12, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

## Load Test Set

#### extension to get filenames
[link](https://gist.github.com/andrewjong/6b02ff237533b3b2c554701fb53d5c4d)

In [8]:
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path


In [9]:
data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(size=64),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    }

In [16]:
file_path = '''..\data\plant-seedlings-classification\\test'''
#data_set = datasets.ImageFolder(file_path,transform=data_transforms['train']) # load sets using ImageFolder
dataset = ImageFolderWithPaths(file_path,transform = data_transforms['train'])

In [18]:
dataset

Dataset ImageFolderWithPaths
    Number of datapoints: 794
    Root location: ..\data\plant-seedlings-classification\test
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(64, 64), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [55]:
testloader = torch.utils.data.DataLoader(dataset, batch_size=794, shuffle=False)

In [56]:
len(testloader)

1

In [57]:
results = []
with torch.no_grad():
            model.eval()
            for image, label,path in testloader:
                output = torch.exp(model(image))
                result = output.numpy()
                classification = np.argmax(result)
                results.append((path,classification))
               

In [58]:
classes = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent', 'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']

In [60]:
output[19]

tensor([0.0576, 0.0775, 0.0614, 0.1263, 0.0501, 0.0983, 0.1346, 0.0438, 0.1085,
        0.0540, 0.1040, 0.0840])

In [61]:
output[1]

tensor([0.0576, 0.0775, 0.0614, 0.1263, 0.0501, 0.0983, 0.1346, 0.0438, 0.1085,
        0.0540, 0.1040, 0.0840])

In [47]:
len(results)

794